# Sentiment Analysis of Airline Reviews

In [1]:

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import spacy
nlp = spacy.load('en_core_web_sm')
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.tokenize import word_tokenize,sent_tokenize
from textblob import TextBlob 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec
from mappings import contraction_mapping,chat_words_replacements

d:\Github\Airline-Review-Sentiment-Analysis\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nltk_sw = nltk.corpus.stopwords.words('english')
spacy_sw = nlp.Defaults.stop_words
stop_words = set(nltk_sw).union(spacy_sw)

# Removing some words from stop words because of sentiment analysis such as no, not
stop_words.remove('no')
stop_words.remove('not')

# Panda display options
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_rows', None)

print(stop_words)
print(len(stop_words))

{'had', 'formerly', 'wherein', "couldn't", 'where', 'herein', 'any', 'whence', '‘s', "'m", 'noone', 'always', "you're", 'hadn', 'nowhere', 'whereas', 'next', 'how', "should've", 'rather', '‘ve', 'latter', 'eleven', 'nor', '’ll', 'thereupon', 'are', 'about', 'show', 'has', 'former', '’re', 'n‘t', 've', 'keep', '‘d', 'none', 'made', 'on', 'myself', 'and', 'fifteen', 'above', 'via', 'side', 'cannot', 'us', 'several', 'as', 'thus', 'than', 'this', 'someone', 'since', 'least', 'seeming', 'alone', 'nine', 'for', "shouldn't", 'herself', 'among', 'below', "aren't", "hadn't", '’s', 'thereby', 'becoming', 'once', 'anyway', 'almost', 'what', 'although', 'his', 'all', 'don', 'ourselves', 'he', 'having', 'but', 'forty', 'theirs', 'sometimes', 'meanwhile', 'its', "don't", 'have', 'toward', 'already', "haven't", 'between', 'further', 'beyond', 'won', 'others', 'without', "'ve", '’m', 'anyhow', 'until', 'thence', 'whom', 'hers', 'haven', 'shan', 'be', 'therein', 'four', 'under', 'part', 'whether', 'd'

In [3]:
df = pd.read_csv('airline_df_cleaned.csv')
reviews = df[['Country','Airline','Review']].copy(deep=True)
reviews.head()

,Country,Airline,Review
0,China,Air China,los angeles to beijing return. food low qualit...
1,China,Air China,round to trip from hong kong to munich. the ma...
2,China,Air China,sydney to beijing to paris then rome to beijin...
3,China,Air China,london to sydney return via beijing. a cheap f...
4,China,Air China,beijing to shanghai. only one check to in desk...


In [4]:
reviews['Review'][60]

'hong kong to beijing to tokyo 12 12 12. first flight a321 cabin crew outstanding although the english speaking fa had a very heavy accent which made it a little difficult to understand her. service very attentive they actually opened the slippers and blanket pouches for you. meal service came and they opened the tray tables for you. breakfast however was watery eggs tasteless chicken sausage and spinach. in the past i have always found catering from hong kong to be outstanding but this was far from what is usual. transit in beijing was good arrived and departed from hard stands business class passengers had their own minibus to take them to and from the terminal. however transit in beijing meant going through an immigration check and there was only one open desk which the immigration officer took 10 mins or more on the passengers ahead of me on line. beijing to tokyo on 738 with only 8 seats. catering was sub par the steak was overcooked and the appetizers were brown tofu and dried fi

In [5]:
# Lemmatizing the words in each review using Spacy

def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if token not in stop_words])

reviews['Review'] = reviews['Review'].apply(lemmatize_text)


In [6]:
def words_only(text):
    return " ".join(re.findall(r'[A-Za-z]+', text))

reviews['Review2'] = reviews['Review'].apply(lambda x: words_only(x))

reviews['Review2'][60]

'hong kong to beijing to tokyo first flight a cabin crew outstanding although the english speak fa have a very heavy accent which make it a little difficult to understand she service very attentive they actually open the slipper and blanket pouch for you meal service come and they open the tray table for you breakfast however be watery egg tasteless chicken sausage and spinach in the past I have always find cater from hong kong to be outstanding but this be far from what be usual transit in beijing be good arrive and depart from hard stand business class passenger have their own minibus to take they to and from the terminal however transit in beijing mean go through an immigration check and there be only one open desk which the immigration officer take min or more on the passenger ahead of I on line beijing to tokyo on with only seat catering be sub par the steak be overcooked and the appetizer be brown tofu and dry fish again though the cabin crew be excellent'

In [7]:
reviews['Review'][60]

'hong kong to beijing to tokyo 12 12 12 . first flight a321 cabin crew outstanding although the english speak fa have a very heavy accent which make it a little difficult to understand she . service very attentive they actually open the slipper and blanket pouch for you . meal service come and they open the tray table for you . breakfast however be watery egg tasteless chicken sausage and spinach . in the past I have always find cater from hong kong to be outstanding but this be far from what be usual . transit in beijing be good arrive and depart from hard stand business class passenger have their own minibus to take they to and from the terminal . however transit in beijing mean go through an immigration check and there be only one open desk which the immigration officer take 10 min or more on the passenger ahead of I on line . beijing to tokyo on 738 with only 8 seat . catering be sub par the steak be overcooked and the appetizer be brown tofu and dry fish . again though the cabin c

In [24]:
# Tokenizing the each review using word_tokenize
corpus = []
for i in range(0, len(reviews)):
    review = word_tokenize(reviews['Review2'][i])
    review = ' '.join(review)
    corpus.append(review)

In [25]:
corpus = [word_tokenize(x)for x in corpus]
corpus

[['los',
  'angeles',
  'to',
  'beijing',
  'return',
  'food',
  'low',
  'quality',
  'staff',
  'appear',
  'when',
  'time',
  'for',
  'mandatory',
  'service',
  'large',
  'screen',
  'view',
  'on',
  'main',
  'bulkhead',
  'without',
  'sound',
  'on',
  'both',
  'trip',
  'they',
  'be',
  'old',
  'it',
  'seem',
  'staff',
  'have',
  'something',
  'to',
  'say',
  'at',
  'irregular',
  'interval',
  'make',
  'it',
  'hard',
  'to',
  'try',
  'and',
  'sleep',
  'but',
  'they',
  'be',
  'very',
  'good',
  'at',
  'make',
  'business',
  'transaction',
  'ignore',
  'everyone',
  'so',
  'they',
  'could',
  'count',
  'money',
  'and',
  'look',
  'at',
  'receipt',
  'you',
  'want',
  'a',
  'cheap',
  'very',
  'basic',
  'airline',
  'trip',
  'to',
  'china',
  'then',
  'this',
  'be',
  'the',
  'airline',
  'for',
  'you'],
 ['round',
  'to',
  'trip',
  'from',
  'hong',
  'kong',
  'to',
  'munich',
  'the',
  'main',
  'reason',
  'for',
  'fly',
  'air

In [26]:
# Building the Word2Vec model

gensim_model = Word2Vec(vector_size=100, window=5)

gensim_model.build_vocab(corpus)

gensim_model.train(corpus, total_examples=gensim_model.corpus_count, epochs=gensim_model.epochs)


(6837015, 9980125)

In [27]:

gensim_model.wv.most_similar('flight')

[('journey', 0.6416610479354858),
 ('sector', 0.5399031639099121),
 ('fight', 0.45173555612564087),
 ('plane', 0.4383367598056793),
 ('trip', 0.43639856576919556),
 ('route', 0.40748268365859985),
 ('segment', 0.3955461382865906),
 ('schedule', 0.39219605922698975),
 ('layover', 0.3911184072494507),
 ('connection', 0.376895934343338)]

In [28]:

gensim_model.wv.most_similar('seat')

[('seating', 0.7759271860122681),
 ('position', 0.623144268989563),
 ('row', 0.6224852800369263),
 ('window', 0.5801792144775391),
 ('bed', 0.5771785974502563),
 ('recline', 0.5687187314033508),
 ('legroom', 0.5552802085876465),
 ('foot', 0.554288387298584),
 ('inch', 0.5527236461639404),
 ('section', 0.5312129259109497)]

In [29]:

gensim_model.wv.most_similar('legroom')

[('pitch', 0.8702948093414307),
 ('width', 0.8314840793609619),
 ('space', 0.7800920009613037),
 ('seating', 0.7611691951751709),
 ('privacy', 0.7213759422302246),
 ('cushion', 0.7194727063179016),
 ('spacious', 0.7159948348999023),
 ('inch', 0.7131651043891907),
 ('roomy', 0.6883421540260315),
 ('design', 0.68730229139328)]

In [30]:

gensim_model.wv.most_similar('turbulence')

[('announcement', 0.6199317574501038),
 ('heavy', 0.5888583660125732),
 ('rain', 0.586529552936554),
 ('runway', 0.5593541264533997),
 ('wind', 0.556833803653717),
 ('freeze', 0.5282845497131348),
 ('confusion', 0.5266417264938354),
 ('captain', 0.5253375172615051),
 ('sign', 0.5249280333518982),
 ('pilot', 0.5204933881759644)]

In [31]:

gensim_model.wv.most_similar('accommodation')

[('voucher', 0.7683131694793701),
 ('hotel', 0.7593238353729248),
 ('compensation', 0.7437990307807922),
 ('transportation', 0.6914553642272949),
 ('further', 0.5793068408966064),
 ('guidance', 0.5754929780960083),
 ('alternative', 0.5728843808174133),
 ('solution', 0.5709404349327087),
 ('notification', 0.570487916469574),
 ('accomodation', 0.5641239285469055)]

In [32]:
gensim_model.wv.most_similar('passenger')

[('people', 0.7845874428749084),
 ('pax', 0.743033230304718),
 ('everyone', 0.6048195958137512),
 ('guest', 0.530924916267395),
 ('woman', 0.5223473310470581),
 ('person', 0.48767563700675964),
 ('traveller', 0.4795440137386322),
 ('curtain', 0.47941023111343384),
 ('traveler', 0.4574500620365143),
 ('those', 0.45338761806488037)]

In [33]:
gensim_model.wv.most_similar('airhostess')

[('sarcastic', 0.757683515548706),
 ('gentle', 0.7441290616989136),
 ('demeanor', 0.7338637113571167),
 ('ineffective', 0.7314476370811462),
 ('positively', 0.7265425324440002),
 ('nature', 0.723784327507019),
 ('empathetic', 0.7202509641647339),
 ('inept', 0.7193183302879333),
 ('attendance', 0.7158530354499817),
 ('spaghetti', 0.7147464156150818)]

In [34]:
gensim_model.wv.most_similar('pilot')

[('captain', 0.8549165725708008),
 ('announcement', 0.6651067733764648),
 ('purser', 0.6001328825950623),
 ('progress', 0.565694272518158),
 ('supervisor', 0.5635995268821716),
 ('seatbelt', 0.5593760013580322),
 ('sign', 0.5575541853904724),
 ('stewardess', 0.5565767884254456),
 ('instruction', 0.5550626516342163),
 ('cockpit', 0.5412799715995789)]

In [35]:
gensim_model.wv.most_similar('airline')

[('carrier', 0.7877746224403381),
 ('company', 0.7619563937187195),
 ('airlines', 0.6529375314712524),
 ('emirate', 0.5722184777259827),
 ('ana', 0.5181614756584167),
 ('airasia', 0.49714308977127075),
 ('spicejet', 0.49004286527633667),
 ('sia', 0.48464757204055786),
 ('traveler', 0.4845580458641052),
 ('aspect', 0.4833831489086151)]

In [36]:
gensim_model.wv.most_similar('aircraft')

[('plane', 0.8751934766769409),
 ('airplane', 0.7913362979888916),
 ('lavatory', 0.6098265051841736),
 ('washroom', 0.6007047295570374),
 ('boeing', 0.5933281779289246),
 ('brand', 0.5914248824119568),
 ('b', 0.5858851075172424),
 ('interior', 0.5835815072059631),
 ('dreamliner', 0.5778257846832275),
 ('bathroom', 0.5747292041778564)]

In [37]:
gensim_model.wv.most_similar('china',topn=20)

[('india', 0.8873516917228699),
 ('korean', 0.7984029650688171),
 ('vietjet', 0.750176191329956),
 ('lion', 0.7418328523635864),
 ('france', 0.7414723634719849),
 ('canada', 0.7360861897468567),
 ('silk', 0.6862483620643616),
 ('oman', 0.6735003590583801),
 ('asia', 0.6608836054801941),
 ('conditioner', 0.6490458846092224),
 ('conditioning', 0.6454830169677734),
 ('bridge', 0.5845644474029541),
 ('alaska', 0.5697926878929138),
 ('freshener', 0.5676854252815247),
 ('eva', 0.5610808730125427),
 ('host', 0.5498482584953308),
 ('hostess', 0.5421949625015259),
 ('gulf', 0.5265655517578125),
 ('wing', 0.5203956365585327),
 ('cs', 0.49306368827819824)]

In [22]:
reviews.to_csv('airline_df_nlped.csv',index=False)